# Library Import

In [1]:
import duckdb
import leafmap

# Sample Data

In [ ]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

# Connecting to DuckDB

In [3]:
con = duckdb.connect('nyc_data.db')

In [4]:
con.install_extension('spatial')
con.load_extension('spatial')

In [5]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

In [6]:
con.sql("SELECT * from nyc_subway_stations LIMIT 5")

┌──────────┬────────┬──────────────┬─────────────────┬───┬─────────┬─────────┬─────────┬──────────────────────┐
│ OBJECTID │   ID   │     NAME     │    ALT_NAME     │ … │  COLOR  │ EXPRESS │ CLOSED  │         geom         │
│  double  │ double │   varchar    │     varchar     │   │ varchar │ varchar │ varchar │       geometry       │
├──────────┼────────┼──────────────┼─────────────────┼───┼─────────┼─────────┼─────────┼──────────────────────┤
│      1.0 │  376.0 │ Cortlandt St │ NULL            │ … │ YELLOW  │ NULL    │ NULL    │ POINT (583521.8544…  │
│      2.0 │    2.0 │ Rector St    │ NULL            │ … │ RED     │ NULL    │ NULL    │ POINT (583324.4866…  │
│      3.0 │    1.0 │ South Ferry  │ NULL            │ … │ RED     │ NULL    │ NULL    │ POINT (583304.1823…  │
│      4.0 │  125.0 │ 138th St     │ Grand Concourse │ … │ GREEN   │ NULL    │ NULL    │ POINT (590250.1059…  │
│      5.0 │  126.0 │ 149th St     │ Grand Concourse │ … │ GREEN   │ express │ NULL    │ POINT (590454.7

# Spatial Relationships

## ST_Equals

`ST_Equals(geometry A, geometry B)` tests the spatial equality of two geometries.

![st_equals](./images/st_equals.png)

`ST_Equals` returns `TRUE` if two geometries of the `same type` have identical `x,y` coordinate `values`, i.e. if the second shape is equal (identical) to the first shape

In [7]:
con.sql(
    """
        SELECT name, geom, ST_AsText(geom)
        FROM nyc_subway_stations
        WHERE name = 'Broad St';
    """
)

┌──────────┬─────────────────────────────────────────────┬─────────────────────────────────────────────┐
│   NAME   │                    geom                     │               st_astext(geom)               │
│ varchar  │                  geometry                   │                   varchar                   │
├──────────┼─────────────────────────────────────────────┼─────────────────────────────────────────────┤
│ Broad St │ POINT (583571.9059213118 4506714.341192182) │ POINT (583571.9059213118 4506714.341192182) │
└──────────┴─────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [8]:
con.sql(
    """
        SELECT name
        FROM nyc_subway_stations
        WHERE ST_Equals(geom, ST_GeomFromText('POINT (583571.9059213118 4506714.341192182)'));
    """
)

┌──────────┐
│   NAME   │
│ varchar  │
├──────────┤
│ Broad St │
└──────────┘

## ST_Intersects, ST_Disjoint, ST_Crosses and ST_Overlaps

`ST_Intersects`, `ST_Crosses`, and `ST_Overlaps` test whether the interiors of the geometries intersect.

![st_intersects](./images/st_intersects.png)

`ST_Intersects(geometry A, geometry B)` returns t (TRUE) if the two shapes have any space in common, i.e., if their boundaries or interiors intersect.

![st_disjoints](./images/st_disjoint.png)

The opposite of ST_Intersects is `ST_Disjoint(geometry A , geometry B)`. If two geometries are disjoint, they do not intersect, and vice-versa. In fact, it is often more efficient to test “not intersects” than to test “disjoint” because the intersects tests can be spatially indexed, while the disjoint test cannot.

![st_crosses](./images/st_crosses.png)

For multipoint/polygon, multipoint/linestring, linestring/linestring, linestring/polygon, and linestring/multipolygon comparisons, `ST_Crosses(geometry A, geometry B)` returns t (TRUE) if the intersection results in a geometry whose dimension is one less than the maximum dimension of the two source geometries and the intersection set is interior to both source geometries.

![st_overlaps](./images/st_overlaps.png)

`ST_Overlaps(geometry A, geometry B)` compares two geometries of the same dimension and returns TRUE if their intersection set results in a geometry different from both but of the same dimension.

In [9]:
con.sql(
    """
        SELECT name, ST_AsText(geom)
        FROM nyc_subway_stations
        WHERE name = 'Broad St';
    """
)

┌──────────┬─────────────────────────────────────────────┐
│   NAME   │               st_astext(geom)               │
│ varchar  │                   varchar                   │
├──────────┼─────────────────────────────────────────────┤
│ Broad St │ POINT (583571.9059213118 4506714.341192182) │
└──────────┴─────────────────────────────────────────────┘

In [10]:
con.sql("FROM nyc_neighborhoods LIMIT 5")

┌───────────┬──────────────────────┬───────────────────────────────────────────────────────────────────────────────────┐
│ BORONAME  │         NAME         │                                       geom                                        │
│  varchar  │       varchar        │                                     geometry                                      │
├───────────┼──────────────────────┼───────────────────────────────────────────────────────────────────────────────────┤
│ Brooklyn  │ Bensonhurst          │ MULTIPOLYGON (((582771.4257198056 4495167.427365481, 584651.2943549604 4497541.…  │
│ Manhattan │ East Village         │ MULTIPOLYGON (((585508.7534890148 4509691.267208001, 586826.3570590394 4508984.…  │
│ Manhattan │ West Village         │ MULTIPOLYGON (((583263.2776595836 4509242.626023987, 583276.8199068634 4509378.…  │
│ The Bronx │ Throggs Neck         │ MULTIPOLYGON (((597640.0090688139 4520272.719938631, 597647.7457808304 4520617.…  │
│ The Bronx │ Wakefield-Williams

In [11]:
con.sql(
    """
        SELECT name, boroname
        FROM nyc_neighborhoods
        WHERE ST_Intersects(geom, ST_GeomFromText('POINT(583571 4506714)'));
    """
)

┌────────────────────┬───────────┐
│        NAME        │ BORONAME  │
│      varchar       │  varchar  │
├────────────────────┼───────────┤
│ Financial District │ Manhattan │
└────────────────────┴───────────┘

## ST_Touches

`ST_Touches` tests whether two geometries touch at their boundaries, but do not intersect in their interiors

![st_touches](./images/st_touches.png)

`ST_Touches(geometry A, geometry B)` returns TRUE if either of the geometries’ boundaries intersect or if only one of the geometry’s interiors intersects the other’s boundary.

## ST_Within and ST_Contains

`ST_Within` and `ST_Contains` test whether one geometry is fully within the other.

![st_within](./images/st_within.png)

`ST_Within(geometry A , geometry B)` returns TRUE if the first geometry is completely within the second geometry. ST_Within tests for the exact opposite result of ST_Contains.

`ST_Contains(geometry A, geometry B)` returns TRUE if the second geometry is completely contained by the first geometry.

## ST_Distance and ST_DWithin

The `ST_Distance(geometry A, geometry B)` calculates the shortest distance between two geometries and returns it as a float. This is useful for actually reporting back the distance between objects

In [12]:
con.sql(
    """
        SELECT ST_Distance(
        ST_GeomFromText('POINT(0 5)'),
        ST_GeomFromText('LINESTRING(-2 2, 2 2)')) as dist;
    """
)

┌────────┐
│  dist  │
│ double │
├────────┤
│    3.0 │
└────────┘

For testing whether two objects are within a distance of one another, the `ST_DWithin` function provides an index-accelerated true/false test. This is useful for questions like “how many trees are within a 500 meter buffer of the road?”. You don’t have to calculate an actual buffer, you just have to test the distance relationship.

![st_dwithin](./images/st_dwithin.png)

In [13]:
con.sql("FROM nyc_streets LIMIT 5")

┌───────┬─────────────┬─────────┬───────────────┬──────────────────────────────────────────────────────────────────────┐
│  ID   │    NAME     │ ONEWAY  │     TYPE      │                                 geom                                 │
│ int32 │   varchar   │ varchar │    varchar    │                               geometry                               │
├───────┼─────────────┼─────────┼───────────────┼──────────────────────────────────────────────────────────────────────┤
│     1 │ Shore Pky S │ NULL    │ residential   │ MULTILINESTRING ((586785.4767897038 4492901.0014554765, 586898.232…  │
│     2 │ NULL        │ NULL    │ footway       │ MULTILINESTRING ((586645.0073625665 4504977.750360583, 586664.2248…  │
│     3 │ Avenue O    │ NULL    │ residential   │ MULTILINESTRING ((586750.3019977848 4496109.72213903, 586837.37268…  │
│     4 │ Walsh Ct    │ NULL    │ residential   │ MULTILINESTRING ((586728.695515043 4497971.05313857, 586886.358225…  │
│     5 │ NULL        │ NULL    

In [14]:
con.sql(
    """
        SELECT name
        FROM nyc_streets
        WHERE ST_DWithin(
            geom,
            ST_GeomFromText('POINT(583571 4506714)'),
            10
        );
    """
)

┌───────────┐
│   NAME    │
│  varchar  │
├───────────┤
│ Wall St   │
│ Broad St  │
│ Nassau St │
└───────────┘

# Function List

- [ST_Contains(geometry A, geometry B)](http://postgis.net/docs/ST_Contains.html): Returns true if and only if no points of B lie in the exterior of A, and at least one point of the interior of B lies in the interior of A.

- [ST_Crosses(geometry A, geometry B)](http://postgis.net/docs/ST_Crosses.html): Returns TRUE if the supplied geometries have some, but not all, interior points in common.

- [ST_Disjoint(geometry A , geometry B)](http://postgis.net/docs/ST_Disjoint.html): Returns TRUE if the Geometries do not “spatially intersect” - if they do not share any space together.

- [ST_Distance(geometry A, geometry B)](http://postgis.net/docs/ST_Distance.html): Returns the 2-dimensional cartesian minimum distance (based on spatial ref) between two geometries in projected units.

- [ST_DWithin(geometry A, geometry B, radius)](http://postgis.net/docs/ST_DWithin.html): Returns true if the geometries are within the specified distance (radius) of one another.

- [ST_Equals(geometry A, geometry B)](http://postgis.net/docs/ST_Equals.html): Returns true if the given geometries represent the same geometry. Directionality is ignored.

- [ST_Intersects(geometry A, geometry B)](http://postgis.net/docs/ST_Intersects.html): Returns TRUE if the Geometries/Geography “spatially intersect” - (share any portion of space) and FALSE if they don’t (they are Disjoint).

- [ST_Overlaps(geometry A, geometry B)](http://postgis.net/docs/ST_Overlaps.html): Returns TRUE if the Geometries share space, are of the same dimension, but are not completely contained by each other.

- [ST_Touches(geometry A, geometry B)](http://postgis.net/docs/ST_Touches.html): Returns TRUE if the geometries have at least one point in common, but their interiors do not intersect.

- [ST_Within(geometry A , geometry B)](http://postgis.net/docs/ST_Within.html): Returns true if the geometry A is completely inside geometry B

In [15]:
df = con.sql("SELECT * EXCLUDE geom, ST_AsText(geom) AS geometry FROM nyc_streets").df()
df.head()

,ID,NAME,ONEWAY,TYPE,geometry
0,1,Shore Pky S,None,residential,MULTILINESTRING ((586785.4767897038 4492901.00...
1,2,None,None,footway,MULTILINESTRING ((586645.0073625665 4504977.75...
2,3,Avenue O,None,residential,MULTILINESTRING ((586750.3019977848 4496109.72...
3,4,Walsh Ct,None,residential,MULTILINESTRING ((586728.695515043 4497971.053...
4,5,None,None,motorway_link,MULTILINESTRING ((586587.0531467082 4510088.25...


In [16]:
gdf = leafmap.df_to_gdf(df, src_crs="EPSG:26918", dst_crs="EPSG:4326")
gdf.head()

,ID,NAME,ONEWAY,TYPE,geometry
0,1,Shore Pky S,None,residential,"MULTILINESTRING ((-73.97454 40.58235, -73.9732..."
1,2,None,None,footway,"MULTILINESTRING ((-73.97454 40.69114, -73.9743..."
2,3,Avenue O,None,residential,"MULTILINESTRING ((-73.97452 40.61126, -73.9734..."
3,4,Walsh Ct,None,residential,"MULTILINESTRING ((-73.97451 40.62802, -73.9726..."
4,5,None,None,motorway_link,"MULTILINESTRING ((-73.97452 40.73718, -73.9738..."


In [ ]:
# !pip install lonboard
leafmap.view_vector(gdf)